# Train the multimodal neural network with a resnet backbone

In [23]:
#import module for multimodal model for air pollution
from models.multimodal_model_resnet import PMModel
# import the module to create the multiModal dataset
from models.data_loader import AirPollutionDataset
from torch.utils.data import DataLoader
import torch
# ramdomsplit is used to divide the dataset as train test split
from torch.utils.data import random_split
from torch import nn
from torch import optim 
from torchvision import  transforms
from torch import random
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from matplotlib import pyplot as plt


In [24]:
if torch.cuda.is_available(): 
 device = "cuda" 
else: 
 device = "cpu"
device

'cpu'

In [25]:
random_seed = 0
torch.manual_seed(random_seed)

In [26]:
#transforms
transform =transforms.Compose([
        #transforms.CenterCrop(10),
        transforms.Resize((224,224)),
        transforms.PILToTensor(),
        transforms.RandomVerticalFlip(p=1)
        #transforms.Normalize((0.485, 0.456, 0.406,0.406), (0.229, 0.224, 0.225,0.225))

        #transforms.ConvertImageDtype(torch.float)
        ]
)


In [27]:
# create the mulyimodal dataset
images_dir='data_dir'
csv_file='multimodal_dataset.csv'


#transform = transforms.Resize(output_size)
dataset = AirPollutionDataset(images_dir,csv_file,transform=transform)


In [28]:
# creation of the dataloader
# Define the proportions for train, validation, and test sets
train_ratio = 0.8# 80% for training
val_ratio = 0.1# 10% for validation
test_ratio = 0.1 # 10% for testing

# Calculate the lengths of each split
total_samples = len(dataset)
train_size = int(train_ratio * total_samples)
val_size = int(val_ratio * total_samples)
test_size = total_samples - train_size - val_size

# Use random_split to create the splits
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoader instances for each split
batch_size = 4
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
print(len(train_loader))
print(len(val_loader))
print(len(test_loader))


56
7
8


In [29]:
# Instantiate your model
model1 = PMModel(S5no2_num_features=10, S5so2_num_features=10,tabular_input_count=5,tabular_features=1,out=1)

# Define loss function and optimizer
l2_lambda = 0.1
criterion = nn.MSELoss()  # For example, you can change this based on your task
optimizer = optim.Adam(model1.parameters(), lr=0.00001)




/Users/armandine/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/Users/armandine/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [30]:
# Training loop
num_epochs =30
val_losses=[]
train_losses=[]
for epoch in range(num_epochs):
    model1.train()
    avg_trainloss=0.0
    total_train_loss=0.0
    for batch_data in train_loader:
       # print(len(batch_data))
        S5no2,S5so2,tabular,labels=batch_data
        S5no2=S5no2.float().to(device)
        S5so2=S5so2.float().to(device)
        tabular=tabular.float().to(device)
        labels=labels.float().reshape(-1,1).to(device)
        #print(S5no2.shape)
        #print(tabular.shape)
        
        
        optimizer.zero_grad()
      
        outputs = model1(S5no2,S5so2,tabular).float()
        #l2_loss = sum(p.norm(2) for p in model1.parameters())
        loss = criterion(outputs, labels)
        #loss = loss + l2_lambda * l2_loss
        total_train_loss+=loss
        
        #loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    avg_trainloss=total_train_loss/len(train_loader)
       
    
    # Validation
    model1.eval()
    with torch.no_grad():
        val_loss = 0.0
        val_r=0.0
        for batch_data in val_loader:
            #print(len(batch_data))
            S5no2,S5so2,tabular,labels=batch_data 
            S5no2=S5no2.float().to(device)
            S5so2=S5so2.float().to(device)
            tabular=tabular.float().to(device)
            labels=labels.float().reshape(-1,1).to(device)
            outputs = model1(S5no2,S5so2,tabular)
            val_loss += criterion(outputs, labels)
            val_r+= r2_score(outputs.cpu(),labels.cpu())
            
        average_val_loss = val_loss / len(val_loader)
        average_val_r=val_r/len(val_loader)
        
        val_losses.append(average_val_loss.item())
        train_losses.append(avg_trainloss.item())
        torch.save(model1.state_dict(), f'models/checkpoints/cnn_backbone/model_epoch_{epoch+1}.pth')
        
    print(f"Epoch [{epoch+1}/{num_epochs}] - Train loss : {avg_trainloss}  - Val Loss: {average_val_loss.item()}  - Rsquare:{average_val_r}")


Epoch [1/30] - Train loss : 623.2091674804688  - Val Loss: 773.0000610351562  - Rsquare:-85393.17730617935
Epoch [2/30] - Train loss : 618.5892944335938  - Val Loss: 766.1903076171875  - Rsquare:-55690.89927977892
Epoch [3/30] - Train loss : 609.8261108398438  - Val Loss: 757.8883056640625  - Rsquare:-39146.15029552828
Epoch [4/30] - Train loss : 601.0767211914062  - Val Loss: 748.7943725585938  - Rsquare:-20663.504129380985
Epoch [5/30] - Train loss : 610.30908203125  - Val Loss: 742.138671875  - Rsquare:-17712.408955674102
Epoch [6/30] - Train loss : 879.0184326171875  - Val Loss: 734.2096557617188  - Rsquare:-12378.682840118025
Epoch [7/30] - Train loss : 580.7213134765625  - Val Loss: 727.0372924804688  - Rsquare:-8516.67271004105
Epoch [8/30] - Train loss : 567.3833618164062  - Val Loss: 711.0931396484375  - Rsquare:-4779.105089934389
Epoch [9/30] - Train loss : 566.7987060546875  - Val Loss: 704.3303833007812  - Rsquare:-3628.7008703512975
Epoch [10/30] - Train loss : 550.5092163

In [58]:
model1.load_state_dict(torch.load('models/checkpoints/resnet_backbone/model_epoch_25.pth'))

<All keys matched successfully>

In [60]:
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error,median_absolute_error
model1.eval()

# Make predictions on the test set
y_true = []
y_pred = []

with torch.no_grad():
    for batch_data in test_loader:
        S5no2,S5so2,tabular,labels=batch_data 
        S5no2=S5no2.float().to(device)
        S5so2=S5so2.float().to(device)
        tabular=tabular.float().to(device)
        labels=labels.float().reshape(-1,1).to(device)
        outputs = model1(S5no2,S5so2,tabular)
        
        y_true.extend(labels.numpy())
        y_pred.extend(outputs.numpy())

# Calculate R-squared score
r2 = r2_score(y_true, y_pred)
mse=mean_squared_error(y_true,y_pred)
mae=mean_absolute_error(y_true,y_pred)
rmse=mean_squared_error(y_true,y_pred)**0.5

print(f"R-squared score: {r2:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

R-squared score: 0.2183
MSE: 369.5028
RMSE: 19.2225
MAE: 9.1935
